simaliar to the first attempt but with cleaner code, more flexible layer creation

In [106]:
import numpy as np
import math
print("start!")


start!


In [107]:
def create_layer(layer_size, dtype=np.float32, **kwargs):
    layer = np.zeros(layer_size, dtype=dtype)
    return(layer)

def create_layer_weights(prev_ly_size, curr_ly_size, init=True):
    if init:
        ly_weights = np.random.rand(prev_ly_size, curr_ly_size)
    else:
        ly_weights = np.zeros(shape=(prev_ly_size, curr_ly_size))
    return(ly_weights)

def create_biases(ly_size, init=True):
    if init:
        biases = np.random.rand(ly_size)
    else:
        biases = np.zeros(ly_size)
    return(biases)


def label_to_output(int):
    outlayer = np.zeros(shape=(10), dtype=np.float32)
    outlayer[int] = 1
    return(outlayer)

class n_network:
    def __init__(self, layers, biases, wieghts):
        self.layers = layers
        self.biases = biases
        self.wieghts = wieghts

def create_network(in_ly_size, out_ly_size, hidden_ly_num, hidden_ly_size):
    in_layer = create_layer(in_ly_size)
    hidden_lys = []
    for index in range(hidden_ly_num):
        hidden_lys.append(create_layer(hidden_ly_size))
    out_layer = create_layer(out_ly_size)
    network_lys = [in_layer, *hidden_lys, out_layer]

    biases =[]
    for i in range(1, len(network_lys), 1):
        biases.append(create_biases(np.size(network_lys[i])))

    weights = []
    for i in range(1, len(network_lys), 1):
        weights.append( create_layer_weights(np.size(network_lys[i-1]), np.size(network_lys[i]), init=True))

    network = n_network(network_lys, biases, weights)
    return(network)


def sigx(number):
    #sigmond function
    try:
        sig_x = ( 1 / (1+ math.exp(-number)))
        #this does not woprk for large negitive numbers 
        # due to rounding errors leading to devide by zero so we'll add error handling
    except OverflowError:
        sig_x = 0
    return(sig_x)

def forward(input_vals, net:n_network):
    #first layer
    net.layers[0] = input_vals

    for index in range(1, len(net.layers), 1):
        net.layers[index] = np.dot(net.layers[(index-1)], net.wieghts[index-1]) + net.biases[index-1]
        
        #activation function
        sigVec = np.vectorize(sigx)
        net.layers[index] = sigVec(net.layers[index])
    return(net)

def node_delta(expected:np.array, actual:np.array):
    delta = expected - actual
    return(delta)



def loss(network_output_ly, expected_ly):
    #loss =  mean of all output neurons - the expected, squared
    loss_vec = (network_output_ly - expected_ly)**2
    loss_sclr = np.sum((network_output_ly - expected_ly)**2) / len(network_output_ly)
    return(loss_sclr, loss_vec)

def backprop_single(net: n_network, expected_out_ly):
    #for i_index, layers in net.layers
    w_delta = np.zeros_like(net.wieghts)
    b_delta = np.zeros_like(net.biases)
    a_delta = np.zeros_like(net.layers)
    print("a_delta:", a_delta)
    print("w_delta:", a_delta)

    first = True
    for l in range(len(net.layers)-1, 0, -1):
        print(len(net.layers))
        print("at l of:", l)

        if first:
                #first(last) layer
            first = False
            a_delta_dir = expected_out_ly - net.layers[l]
            print("delta a, first layer:", a_delta_dir)
            w_delta[l-1] = np.outer(a_delta_dir, net.layers[l])     

        else:
            a_delta_dir = a_delta[l] - net.layers[l]
            print("delta a:", a_delta_dir)
            w_delta[l-1,:,:] = np.outer(a_delta_dir, net.layers[l])


    return(w_delta, b_delta, a_delta)



In [108]:
net = create_network(3,3,1,3)
input = [0,1,2]
output = [0,1,0]
backprop_single(net, output)

a_delta: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
w_delta: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
3
at l of: 2
delta a, first layer: [0. 1. 0.]
3
at l of: 1
delta a: [0. 0. 0.]


(array([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]),
 array([[0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=float32))

In [109]:
network = create_network(3,3,1,3)

input = [0,1,2]
output = [0,1,0]


network = forward(input,network)
#print("layers: ")
#print(network.layers[:])
print("weights: ")
for bunchs in network.wieghts:
    print(bunchs)
w_delta = np.zeros_like(network.wieghts)
print("weights zeros: ")
for bunchs in w_delta:
    print(bunchs)

#print("ouput:",network.layers[-1])

#print(loss(network.layers[-1], output))

print(network.layers[1])

print(network.layers[1])

weights: 
[[0.36558628 0.50312638 0.38493594]
 [0.92157084 0.09648922 0.69928546]
 [0.88231203 0.97478797 0.26162829]]
[[0.2091979  0.11025822 0.18700941]
 [0.02933415 0.81178509 0.4023575 ]
 [0.81371026 0.67059237 0.68730767]]
weights zeros: 
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0.96081732 0.93168407 0.90205332]
[0.96081732 0.93168407 0.90205332]
